In [2]:
print('Yoojiyoung test')

Yoojiyoung test


In [1]:
print('test')

test


In [1]:
from pandas_datareader import data as web

In [3]:
import datetime

In [4]:
start = datetime.datetime(1998, 1, 1)
end = datetime.datetime(2021, 12, 31)
data = web.DataReader('DGS10', 'fred', start, end).dropna()

In [5]:
data

,DGS10
DATE,
1998-01-02,5.67
1998-01-05,5.52
1998-01-06,5.49
1998-01-07,5.55
1998-01-08,5.49
...,...
2021-12-27,1.48
2021-12-28,1.49
2021-12-29,1.55


In [6]:
# Convert to monthly data
monthly_data = data.resample('M').last()

# Calculate monthly returns
monthly_returns = (1 + monthly_data / 100) ** (1/12) - 1

# Calculate cumulative returns
cumulative_returns = (1 + monthly_returns).cumprod()

# You might want to subtract 1 to get the cumulative return in percentage
cumulative_returns = cumulative_returns - 1
cumulative_returns

,DGS10
DATE,
1998-01-31,0.004495
1998-02-28,0.009083
1998-03-31,0.013731
1998-04-30,0.018409
1998-05-31,0.023011
...,...
2021-08-31,1.208215
2021-09-30,1.210993
2021-10-31,1.213829
